# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [41]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [42]:
csv_weather_path = "../WeatherPy/weather_search.csv"
weather_df = pd.read_csv(csv_weather_path)

In [43]:
weather_df

,city,city_id,latitude,longitude,temperature,max_temp,cloudiness,humidity,wind_speed
0,Salta,3838233,-24.7859,-65.4117,64.18,64.40,40,88,4.61
1,New Norfolk,2155415,-42.7826,147.0587,56.21,60.01,96,73,4.00
2,Saskylakh,2017155,71.9167,114.0833,33.57,33.57,100,97,20.42
3,Ushuaia,3833367,-54.8000,-68.3000,42.80,42.80,75,93,14.97
4,Sisimiut,3419842,66.9395,-53.6735,31.78,31.78,100,70,3.31
...,...,...,...,...,...,...,...,...,...
562,Agadez,2448083,19.7500,10.2500,83.95,83.95,20,14,10.40
563,Batticaloa,1250161,7.7102,81.6924,81.73,81.73,8,76,1.57
564,Yilan,1674199,24.7570,121.7530,78.69,81.00,47,95,6.78
565,Vestmannaeyjar,3412093,63.4427,-20.2734,46.20,46.20,100,91,17.25


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:
gmaps.configure(api_key=g_key)

locations = weather_df[["latitude","longitude"]].astype(float)
humidity=weather_df["humidity"].astype(float)

In [45]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=100,point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [74]:
vacation_cities=weather_df.loc[(weather_df['max_temp']>79)&(weather_df['max_temp']<100)&(weather_df['humidity']<50)]['city']
vacation_city_id=weather_df.loc[(weather_df['max_temp']>79)&(weather_df['max_temp']<100)&(weather_df['humidity']<50)]['city_id']
vacation_lat=weather_df.loc[(weather_df['max_temp']>79)&(weather_df['max_temp']<100)&(weather_df['humidity']<50)]['latitude']
vacation_lon=weather_df.loc[(weather_df['max_temp']>79)&(weather_df['max_temp']<100)&(weather_df['humidity']<50)]['longitude']
vacation_temp=weather_df.loc[(weather_df['max_temp']>79)&(weather_df['max_temp']<100)&(weather_df['humidity']<50)]['temperature']
vacation_max_temp=weather_df.loc[(weather_df['max_temp']>79)&(weather_df['max_temp']<100)&(weather_df['humidity']<50)]['max_temp']
vacation_clouds=weather_df.loc[(weather_df['max_temp']>79)&(weather_df['max_temp']<100)&(weather_df['humidity']<50)]['cloudiness']
vacation_humidity=weather_df.loc[(weather_df['max_temp']>79)&(weather_df['max_temp']<100)&(weather_df['humidity']<50)]['humidity']
vacation_wind_speed=weather_df.loc[(weather_df['max_temp']>79)&(weather_df['max_temp']<100)&(weather_df['humidity']<50)]['wind_speed']

vacation_dict={'city':vacation_cities,
              'city_id': vacation_city_id,
              'lat':vacation_lat,
              'lon': vacation_lon,
              'temp':vacation_temp,
              'max_temp': vacation_max_temp,
              'coulds': vacation_clouds,
              'humidity': vacation_humidity,
              'wind_speed':vacation_wind_speed}
vacation_df=pd.DataFrame(vacation_dict)

In [75]:
vacation_df

,city,city_id,lat,lon,temp,max_temp,coulds,humidity,wind_speed
39,Carnarvon,2074865,-24.8667,113.6333,82.40,82.40,2,32,5.75
58,Tura,1254046,25.5198,90.2201,84.97,84.97,1,25,5.12
83,Mount Isa,2065594,-20.7333,139.5000,84.20,84.20,22,28,13.80
138,Tessalit,2449893,20.1986,1.0114,83.88,83.88,0,8,6.69
169,Kununurra,2068110,-15.7667,128.7333,86.00,86.00,14,23,16.11
221,Port Hedland,2063042,-20.3167,118.5667,91.40,91.40,0,15,19.57
228,Celestún,3531368,20.8667,-90.4000,86.90,86.90,96,47,13.24
233,Daura,2345096,11.5538,11.4085,81.19,81.19,46,27,7.52
240,Blackwater,2175403,-23.5833,148.8833,79.83,79.83,20,21,10.18
270,Katherine,2068655,-14.4667,132.2667,84.20,84.20,93,21,11.50


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
